In [27]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import pandas as pd
#from openai import OpenAI
import openai
import re

In [27]:
df = pd.read_csv('DF_pilot_withAutomaticEvaluation.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer,System_evaluation,System_evaluation_foil,Double_check
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,"In the video, the girl is seen texting on her...","No, the girl is not texting a message with her...",Correct,Contradiction,ok
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,"During an earthquake, it's better for people ...","If there was an earthquake, the computer monit...",Not_correct,Contradiction,not_ok
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...,"No, the girl is not using the computer for the...",Not_correct,Not_contradiction,ok
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is perfectly visible.,The weather is sunny.,Not_correct,Not_contradiction,ok
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is disappointed and unsatisfied with...,The girl looked at the woman in disbelief.,Not_correct,Not_contradiction,ok
...,...,...,...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The girl without the axe stays down.,The boy with the axe falls to the ground.,Not_correct,Contradiction,not_ok
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the sword misses the boy with th...,The boy with the sword manages to hit the boy ...,Correct,Contradiction,ok
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","Before being hit by the sword, the boy with t...",The boy with the sword and the boy with the ax...,Not_correct,Not_contradiction,ok
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.,"Yes, the guys are playing.",Correct,Contradiction,ok


In [7]:
df.loc[(df['Double_check']=='ok') & (df['Category']=='Outofscope')].shape[0]

30

In [8]:
df.loc[(df['Double_check']=='not_ok') & (df['Category']=='Outofscope')].shape[0]

6

In [13]:
df.loc[(df['System_evaluation']=='Correct') & (df['Category']=='Outofscope')].shape[0]

6

In [14]:
df.loc[(df['System_evaluation']=='Not_correct') & (df['Category']=='Outofscope')].shape[0]

30

In [9]:
df.loc[(df['Double_check']=='ok') & (df['Category']=='Counterfactual')].shape[0]

12

In [10]:
df.loc[(df['Double_check']=='not_ok') & (df['Category']=='Counterfactual')].shape[0]

21

In [15]:
df.loc[(df['System_evaluation']=='Correct') & (df['Category']=='Counterfactual')].shape[0]

9

In [16]:
df.loc[(df['System_evaluation']=='Not_correct') & (df['Category']=='Counterfactual')].shape[0]

24

In [11]:
df.loc[(df['Double_check']=='ok') & (df['Category']=='Uncertainty')].shape[0]

26

In [12]:
df.loc[(df['Double_check']=='not_ok') & (df['Category']=='Uncertainty')].shape[0]

12

In [24]:
df.loc[(df['System_evaluation']=='Correct') & (df['Category']=='Uncertainty')].shape[0]

3

In [25]:
df.loc[(df['System_evaluation']=='Not_correct') & (df['Category']=='Uncertainty')].shape[0]

34

In [20]:
df.loc[(df['Double_check']=='ok') & (df['Category']=='Entity tracking')].shape[0]

20

In [21]:
df.loc[(df['Double_check']=='not_ok') & (df['Category']=='Entity tracking')].shape[0]

13

In [22]:
df.loc[(df['System_evaluation']=='Correct') & (df['Category']=='Entity tracking')].shape[0]

7

In [23]:
df.loc[(df['System_evaluation']=='Not_correct') & (df['Category']=='Entity tracking')].shape[0]

26

In [24]:
df = pd.read_csv('DF_pilot_withCaptionFoils.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,"In the video, the girl is seen texting on her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,"During an earthquake, it's better for people ..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is perfectly visible.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is disappointed and unsatisfied with...
...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The girl without the axe stays down.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the sword misses the boy with th...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","Before being hit by the sword, the boy with t..."
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.


In [28]:
df1 = pd.read_csv('DF_pilot_withCaption_Foil_Idefics.csv', index_col = 0)
df1

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,"In the video, the girl is seen texting on her...","No, the girl is not texting a message with her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,"During an earthquake, it's better for people ...","If there was an earthquake, the computer monit..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...,"No, the girl is not using the computer for the..."
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is perfectly visible.,The weather is sunny.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is disappointed and unsatisfied with...,The girl looked at the woman in disbelief.
...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The girl without the axe stays down.,The boy with the axe falls to the ground.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the sword misses the boy with th...,The boy with the sword manages to hit the boy ...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","Before being hit by the sword, the boy with t...",The boy with the sword and the boy with the ax...
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.,"Yes, the guys are playing."


In [27]:
df['Idefics_Answer'] = df1['Idefics_Answer']

In [28]:
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,"In the video, the girl is seen texting on her...","No, the girl is not texting a message with her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,"During an earthquake, it's better for people ...","If there was an earthquake, the computer monit..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...,"No, the girl is not using the computer for the..."
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is perfectly visible.,The weather is sunny.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is disappointed and unsatisfied with...,The girl looked at the woman in disbelief.
...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The girl without the axe stays down.,The boy with the axe falls to the ground.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the sword misses the boy with th...,The boy with the sword manages to hit the boy ...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","Before being hit by the sword, the boy with t...",The boy with the sword and the boy with the ax...
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.,"Yes, the guys are playing."


In [29]:
df.to_csv('DF_pilot_withCaption_Foil_Idefics.csv')

In [32]:
openai.api_key = "ca72e43668824cd090e6ea210fe3cd3b"
openai.api_base = "https://hlt-nlp.openai.azure.com/" # your endpoint should look like the following
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
deployment_name='gpt-35-turbo'



def GPT_answer_evali(question, caption, sentence):
    response = openai.ChatCompletion.create(
        engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
        messages=[
            {"role": "system", "content": f"You are an assistant designed to judge the correctness of sentences related to videos. Users paste into a string of text a question Q about a video, an answer A, representing the caption of a video, and a sentence S. You have to judge whether S is correct with reference to Q and A. If so, you must answer \"Correct\", otherwise type \"Not_correct\". "},
            {"role": "user", "content": f"Q: {question}\nA: {caption}\nS{sentence}"}
        ]
    )
    return response['choices'][0]['message']['content']

In [36]:
r = GPT_answer_evali(df1.iloc[339,3], df1.iloc[339,5], df1.iloc[339,7])
r

'Not_correct.'

In [8]:
def create_captionGPT(question, answer):
    response = openai.ChatCompletion.create(
        engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
        messages=[
            {"role": "system", "content": f"You are an assistant designed to create short and concise declarative sentences." 
            f"Users will paste in a string of text a question Q and an answer A which are related to a video scene. Based on Q and A, you have to create "
            f"a sentence that is the caption for the video. Remember: the caption must be short and can contain only what is said in Q and A!"},
            {"role": "user", "content": f"Q: {question}\nA: {answer}"}
        ]
    )
    return response


In [6]:
df.iloc[4,4]

'She seems to be happy and satisfied'

In [9]:
r = create_captionGPT(df.iloc[4,3], df.iloc[4,4])
r

<OpenAIObject chat.completion id=chatcmpl-8mrmDjHdzezfi9c43N2iVFqwxJGQY at 0x7f74b2b3e540> JSON: {
  "id": "chatcmpl-8mrmDjHdzezfi9c43N2iVFqwxJGQY",
  "object": "chat.completion",
  "created": 1706657153,
  "model": "gpt-35-turbo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\"The girl appears happy with the woman's claim.\""
      }
    }
  ],
  "usage": {
    "prompt_tokens": 110,
    "completion_tokens": 10,
    "total_tokens": 120
  }
}

In [10]:
r['choices'][0]['message']['content']

'"The girl appears happy with the woman\'s claim."'

In [11]:
r.choices

[<OpenAIObject at 0x7f74d2e9c630> JSON: {
   "finish_reason": "stop",
   "index": 0,
   "message": {
     "role": "assistant",
     "content": "\"The girl appears happy with the woman's claim.\""
   }
 }]

In [14]:
r.choices[0].message.content

'"The girl appears happy with the woman\'s claim."'

In [20]:
df = pd.read_csv('DF_pilot.csv', index_col = 0)
df

,ID_Video,Category,Annotator,Question,Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied
...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.


In [23]:
df.iloc[336,4]

"It's difficult to say."

In [28]:
openai.api_key = "ca72e43668824cd090e6ea210fe3cd3b"
openai.api_base = "https://hlt-nlp.openai.azure.com/" # your endpoint should look like the following
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
deployment_name='gpt-35-turbo'

def create_captionGPT(question, answer):
    response = openai.ChatCompletion.create(
        engine= deployment_name, # The deployment name you chose when you deployed the GPT-3.5-Turbo or GPT-4 model.
        messages=[
            {"role": "system", "content": f"You are an assistant designed to create declarative sentences." 
            f"Users will paste in a string of text a question Q and an answer A which are related to a video scene. Based on Q and A, you have to create "
            f"a sentence that is the caption for the video. Remember: the caption is a declarative sentence but it must be short and can contain only what is said in Q and A!"},
            {"role": "user", "content": f"Q: {question}\nA: {answer}"}
        ]
    )
    return response.choices[0].message.content

In [29]:
r = create_captionGPT(df.iloc[336,3], df.iloc[336,4])
r

"The boys' activities are uncertain."

In [ ]:
df.read_

In [3]:
df = pd.read_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaption_Foil_Idefics.csv', index_col =0)
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,"In the video, the girl is seen texting on her...","No, the girl is not texting a message with her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,"During an earthquake, it's better for people ...","If there was an earthquake, the computer monit..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...,"No, the girl is not using the computer for the..."
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is perfectly visible.,The weather is sunny.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is disappointed and unsatisfied with...,The girl looked at the woman in disbelief.
...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The girl without the axe stays down.,The boy with the axe falls to the ground.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the sword misses the boy with th...,The boy with the sword manages to hit the boy ...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","Before being hit by the sword, the boy with t...",The boy with the sword and the boy with the ax...
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.,"Yes, the guys are playing."


In [10]:
df.iloc[40,3]

"What door does the key around the teacher's neck open?"

In [11]:
df = df.drop([40])
df

,ID_Video,Category,Annotator,Question,Answer,Caption,Foil,Idefics_Answer
0,8166472@N03_4977206807_49f8b91a19.avi,Outofscope,Davide,Is the girl texting a message with her phone?,There is no mobile phone in the video.,The video does not show the girl using a phone...,"In the video, the girl is seen texting on her...","No, the girl is not texting a message with her..."
1,8166472@N03_4977206807_49f8b91a19.avi,Counterfactual,Davide,What would happen if there was an earthquake?,The people should protect themselves by going ...,The people can protect themselves during an ea...,"During an earthquake, it's better for people ...","If there was an earthquake, the computer monit..."
2,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,Is the girl using the computer for the first t...,It's hard to say,The girl's experience with the computer is unc...,The girl's experience with the computer is ce...,"No, the girl is not using the computer for the..."
3,8166472@N03_4977206807_49f8b91a19.avi,Uncertainty,Davide,What is the weather like?,It can't be seen what's the weather like outside,The weather outside is not visible.,The weather outside is perfectly visible.,The weather is sunny.
4,8166472@N03_4977206807_49f8b91a19.avi,Sentiment,Davide,How did the girl react to the woman's claim?,She seems to be happy and satisfied,The girl is happy and satisfied with the woman...,The girl is disappointed and unsatisfied with...,The girl looked at the woman in disbelief.
...,...,...,...,...,...,...,...,...
339,61082531@N07_7638997290_b024d278f1.mov,Entity tracking,Alessio,What does the boy with the axe do after being ...,He gets back up.,The boy with the axe gets back up.,The girl without the axe stays down.,The boy with the axe falls to the ground.
340,61082531@N07_7638997290_b024d278f1.mov,Temporal,Alessio,How long after getting up does the boy with th...,After about 10 seconds.,The boy with the sword manages to hit the boy ...,The boy with the sword misses the boy with th...,The boy with the sword manages to hit the boy ...
341,61082531@N07_7638997290_b024d278f1.mov,Causal,Alessio,What happens after the boy with the sword hits...,The boy with the axe exults.,"After being hit by the sword, the boy with the...","Before being hit by the sword, the boy with t...",The boy with the sword and the boy with the ax...
342,61082531@N07_7638997290_b024d278f1.mov,Commonsense,Alessio,Are the guys playing?,Yes.,The guys are playing.,The guys are not playing.,"Yes, the guys are playing."


In [12]:
df.to_csv('/home/dtesta/MyProjects/Benchmark_PilotTest/DF_pilot_withCaption_Foil_Idefics.csv')

In [ ]:
def humanEval_check(df):
    for index, row in df.iterrows():
        if row['System_evaluation'] == row['Human_evaluation']:
            df.loc[index, 'Check'] = 'Match'
        else
            df.loc[index, 'Check'] = 'Not_Match'
    

In [ ]:
def match_count(df):
    Match_count = df['Check'].value_counts()['Match']
    match_perc = round((Match_count/df.shape[0])*100, 2)
return
